In [1]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import random
import copy
import os
import graphviz
import pickle
import scipy.io.wavfile as wav
from src.voice_activity_detection.extract_features import extract_features

In [2]:
with open("src/data/noise-train/features_df_1s.pickle", "rb") as file:
    voice_noise_df=pickle.load(file)
voice_noise_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392416 entries, 0 to 392415
Data columns (total 13 columns):
RMS       392416 non-null float64
SE        392416 non-null float64
ZCR       392416 non-null float64
LEFR      392416 non-null float64
SF        392416 non-null float64
SRF       392416 non-null float64
SC        392416 non-null float64
BW        392416 non-null float64
NWPD      392416 non-null float64
RSE       391352 non-null float64
type      392416 non-null object
name      392416 non-null object
number    392416 non-null int64
dtypes: float64(10), int64(1), object(2)
memory usage: 38.9+ MB


In [3]:
voice_noise_df = voice_noise_df[pd.notnull(voice_noise_df['RSE'])]
voice_noise_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 391352 entries, 0 to 392415
Data columns (total 13 columns):
RMS       391352 non-null float64
SE        391352 non-null float64
ZCR       391352 non-null float64
LEFR      391352 non-null float64
SF        391352 non-null float64
SRF       391352 non-null float64
SC        391352 non-null float64
BW        391352 non-null float64
NWPD      391352 non-null float64
RSE       391352 non-null float64
type      391352 non-null object
name      391352 non-null object
number    391352 non-null int64
dtypes: float64(10), int64(1), object(2)
memory usage: 41.8+ MB


In [6]:
voice_noise_df.describe()

,RMS,SE,ZCR,LEFR,SF,SRF,SC,BW,NWPD,RSE,number
count,391352.000000,3.913520e+05,391352.000000,391352.000000,391352.000000,391352.000000,391352.000000,3.913520e+05,391352.000000,3.913520e+05,391352.000000
mean,3881.714478,inf,0.106271,0.295517,0.018697,4468.967769,767.033640,7.007102e+05,0.268781,-inf,216.592666
std,2670.031736,NaN,0.061622,0.240648,0.009159,952.113839,522.916567,7.111859e+05,2.625039,NaN,174.390637
min,0.490089,2.839631e+03,0.000000,0.000000,0.001260,1718.750000,55.003685,6.713368e+03,-60.607956,-inf,0.000000
25%,1947.646912,3.261980e+03,0.065317,0.040816,0.012595,3645.408163,442.476153,2.559816e+05,-1.168530,-4.094069e-01,74.000000
50%,3232.495117,3.406045e+03,0.094693,0.295918,0.017267,4294.961735,645.304517,4.875605e+05,0.315362,-3.534590e-01,171.000000
75%,5282.676758,3.587672e+03,0.133633,0.479592,0.023069,5146.364796,945.288673,8.979258e+05,1.780660,-2.945793e-01,329.000000
max,27775.876953,inf,0.865992,1.000000,0.105078,7559.948980,7029.272341,8.526785e+06,52.321439,-1.398323e-01,1046.000000


In [7]:
voice_noise_df[voice_noise_df['RSE']==-np.inf].describe()

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4406: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above


,RMS,SE,ZCR,LEFR,SF,SRF,SC,BW,NWPD,RSE,number
count,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,1.300000e+01,13.000000,13.000000,13.000000
mean,3608.638606,3007.364822,0.102011,0.412088,0.017316,6008.217229,2103.561929,2.498963e+06,-1.300322,-inf,85.307692
std,4759.159440,155.007518,0.138847,0.309099,0.019586,1001.054050,1493.711665,2.007211e+06,5.852403,NaN,94.908188
min,0.500250,2887.367726,0.000063,0.000000,0.001962,4536.033163,124.978703,1.831095e+05,-18.229590,-inf,0.000000
25%,596.431213,2918.285639,0.035502,0.142857,0.003649,5411.989796,959.586219,9.103380e+05,-0.939346,NaN,0.000000
50%,1663.261963,2928.192392,0.048816,0.397959,0.007498,5860.650510,1722.533033,1.998010e+06,-0.543403,NaN,39.000000
75%,6026.494141,3029.796779,0.101381,0.683673,0.030468,6973.214286,3158.577783,3.961565e+06,0.410266,NaN,156.000000
max,17151.566406,3400.058417,0.531971,0.867347,0.066267,7407.206633,4461.149107,5.951816e+06,8.619842,-inf,297.000000


In [8]:
voice_noise_df[voice_noise_df['SE']==np.inf].describe()

,RMS,SE,ZCR,LEFR,SF,SRF,SC,BW,NWPD,RSE,number
count,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000e+00,8.000000,8.000000,8.000000
mean,3175.613842,inf,0.087466,0.406888,0.014930,5566.924426,1328.194008,1.543237e+06,1.704269,-0.356290,253.750000
std,4477.458490,NaN,0.037786,0.337923,0.009319,1163.421481,1094.219484,1.611028e+06,2.468270,0.074217,176.239083
min,0.502867,inf,0.031377,0.000000,0.002001,3590.561224,514.765713,2.334259e+05,-2.863788,-0.482725,0.000000
25%,502.010110,inf,0.058551,0.030612,0.007868,4898.915816,865.003654,7.119532e+05,0.245167,-0.389377,174.750000
50%,2215.343262,inf,0.095193,0.551020,0.016001,5553.571429,1006.734410,8.580951e+05,2.265976,-0.355277,257.500000
75%,3010.830811,inf,0.109460,0.622449,0.021893,6297.911352,1166.487091,1.785885e+06,3.427412,-0.286524,401.500000
max,13712.734375,inf,0.143821,0.826531,0.026917,7383.290816,3971.037263,5.263086e+06,4.615741,-0.277197,456.000000


In [9]:
voice_noise_df['RSE'].replace(-np.inf, np.nan, inplace=True)
voice_noise_df['SE'].replace(np.inf, np.nan, inplace=True)
voice_noise_df = voice_noise_df[pd.notnull(voice_noise_df['RSE'])]
voice_noise_df = voice_noise_df[pd.notnull(voice_noise_df['SE'])]
voice_noise_df.describe()

,RMS,SE,ZCR,LEFR,SF,SRF,SC,BW,NWPD,RSE,number
count,391331.000000,391331.000000,391331.000000,391331.000000,391331.000000,391331.000000,391331.000000,3.913310e+05,391331.000000,391331.000000,391331.000000
mean,3881.737985,3443.799752,0.106271,0.295511,0.018697,4468.894189,766.977768,7.006333e+05,0.268804,-0.356949,216.596268
std,2669.903785,267.732251,0.061618,0.240642,0.009159,952.056253,522.781783,7.109997e+05,2.624865,0.076928,174.391209
min,0.490089,2839.630946,0.000000,0.000000,0.001260,1718.750000,55.003685,6.713368e+03,-60.607956,-0.792863,0.000000
25%,1947.676636,3261.989045,0.065317,0.040816,0.012595,3645.408163,442.472777,2.559732e+05,-1.168542,-0.409401,74.000000
50%,3232.514160,3406.046937,0.094693,0.295918,0.017267,4294.642857,645.296802,4.875252e+05,0.315362,-0.353456,171.000000
75%,5282.680664,3587.667239,0.133665,0.479592,0.023069,5146.364796,945.229214,8.978401e+05,1.780668,-0.294575,329.000000
max,27775.876953,6964.074221,0.865992,1.000000,0.105078,7559.948980,7029.272341,8.526785e+06,52.321439,-0.139832,1046.000000


In [10]:
le=LabelEncoder()
voice_noise_df['type'] = le.fit_transform(voice_noise_df["type"])
list(le.classes_)


['music', 'noise', 'speech']

In [11]:
voice_noise_df.describe()

,RMS,SE,ZCR,LEFR,SF,SRF,SC,BW,NWPD,RSE,type,number
count,391331.000000,391331.000000,391331.000000,391331.000000,391331.000000,391331.000000,391331.000000,3.913310e+05,391331.000000,391331.000000,391331.000000,391331.000000
mean,3881.737985,3443.799752,0.106271,0.295511,0.018697,4468.894189,766.977768,7.006333e+05,0.268804,-0.356949,1.163263,216.596268
std,2669.903785,267.732251,0.061618,0.240642,0.009159,952.056253,522.781783,7.109997e+05,2.624865,0.076928,0.957945,174.391209
min,0.490089,2839.630946,0.000000,0.000000,0.001260,1718.750000,55.003685,6.713368e+03,-60.607956,-0.792863,0.000000,0.000000
25%,1947.676636,3261.989045,0.065317,0.040816,0.012595,3645.408163,442.472777,2.559732e+05,-1.168542,-0.409401,0.000000,74.000000
50%,3232.514160,3406.046937,0.094693,0.295918,0.017267,4294.642857,645.296802,4.875252e+05,0.315362,-0.353456,2.000000,171.000000
75%,5282.680664,3587.667239,0.133665,0.479592,0.023069,5146.364796,945.229214,8.978401e+05,1.780668,-0.294575,2.000000,329.000000
max,27775.876953,6964.074221,0.865992,1.000000,0.105078,7559.948980,7029.272341,8.526785e+06,52.321439,-0.139832,2.000000,1046.000000


In [12]:
voice_noise_df.groupby('type').count()

,RMS,SE,ZCR,LEFR,SF,SRF,SC,BW,NWPD,RSE,name,number
type,,,,,,,,,,,,
0,152824,152824,152824,152824,152824,152824,152824,152824,152824,152824,152824,152824
1,21793,21793,21793,21793,21793,21793,21793,21793,21793,21793,21793,21793
2,216714,216714,216714,216714,216714,216714,216714,216714,216714,216714,216714,216714


In [13]:
voice_noise_df[voice_noise_df["type"]==0].describe()

,RMS,SE,ZCR,LEFR,SF,SRF,SC,BW,NWPD,RSE,type,number
count,152824.000000,152824.000000,152824.000000,152824.000000,152824.000000,152824.000000,152824.000000,1.528240e+05,152824.000000,152824.000000,152824.0,152824.000000
mean,5260.817023,3420.372148,0.094214,0.105163,0.020546,4709.068245,613.892509,6.933769e+05,0.128544,-0.299098,0.0,144.842237
std,2985.740426,250.963428,0.052359,0.150748,0.010575,996.237397,402.765101,7.257606e+05,2.638707,0.049296,0.0,123.199161
min,0.493140,2869.707977,0.000000,0.000000,0.001876,2520.408163,56.698040,3.292752e+04,-33.636531,-0.650863,0.0,0.000000
25%,3051.106201,3251.670047,0.058066,0.000000,0.013206,3905.931122,370.924919,1.694402e+05,-1.356123,-0.327676,0.0,58.000000
50%,4845.181885,3381.916691,0.083755,0.030612,0.018439,4700.573980,528.047692,4.507839e+05,0.206767,-0.292903,0.0,120.000000
75%,7288.141235,3544.529586,0.121445,0.163265,0.025514,5499.681122,751.463027,9.739027e+05,1.706160,-0.263077,0.0,195.000000
max,20044.730469,5524.586486,0.847928,0.969388,0.105078,7549.426020,6859.332631,7.946042e+06,24.933125,-0.140178,0.0,986.000000


In [14]:
voice_noise_df[voice_noise_df["type"]==1].describe()

,RMS,SE,ZCR,LEFR,SF,SRF,SC,BW,NWPD,RSE,type,number
count,21793.000000,21793.000000,21793.000000,21793.000000,21793.000000,21793.000000,21793.000000,2.179300e+04,21793.000000,21793.000000,21793.0,21793.000000
mean,3262.442013,3282.896360,0.145972,0.148096,0.013333,5329.402764,1118.572892,1.247523e+06,0.423182,-0.295885,1.0,45.461846
std,3442.484476,254.233357,0.134183,0.247799,0.012038,1095.258156,1145.579997,1.402201e+06,4.140876,0.047645,0.0,66.961740
min,0.490089,2839.630946,0.000000,0.000000,0.001260,1718.750000,55.003685,6.713368e+03,-60.607956,-0.792863,1.0,0.000000
25%,859.682983,3122.185949,0.052878,0.000000,0.005402,4462.691327,331.925093,2.624718e+05,-0.760528,-0.309761,1.0,7.000000
50%,2083.710693,3265.899912,0.102194,0.000000,0.009282,5305.803571,677.369560,6.312266e+05,0.503139,-0.287430,1.0,21.000000
75%,4792.392578,3399.129917,0.199637,0.204082,0.017661,6279.655612,1521.557227,1.730447e+06,1.880179,-0.277491,1.0,53.000000
max,27775.876953,6964.074221,0.865992,1.000000,0.093103,7559.948980,7029.272341,8.526785e+06,52.321439,-0.139832,1.0,493.000000


In [15]:
voice_noise_df[voice_noise_df["type"]==2].describe()

,RMS,SE,ZCR,LEFR,SF,SRF,SC,BW,NWPD,RSE,type,number
count,216714.000000,216714.000000,216714.000000,216714.000000,216714.000000,216714.000000,216714.000000,2.167140e+05,216714.000000,216714.000000,216714.0,216714.000000
mean,2971.505799,3476.501229,0.110782,0.444567,0.017933,4212.992756,839.574782,6.507546e+05,0.352189,-0.403886,2.0,284.405779
std,1784.846521,272.879869,0.053206,0.180042,0.007222,797.251688,461.437169,5.577542e+05,2.404989,0.061694,0.0,180.523615
min,0.493964,2891.177680,0.000000,0.000000,0.001935,2725.765306,64.236217,3.821430e+04,-23.636608,-0.707460,2.0,0.000000
25%,1718.941772,3287.646808,0.072255,0.326531,0.012931,3578.125000,520.773055,3.029473e+05,-1.085969,-0.442553,2.0,129.000000
50%,2640.561035,3441.451222,0.101694,0.438776,0.017044,3953.125000,742.857868,4.949136e+05,0.371097,-0.398539,2.0,270.000000
75%,3820.910828,3632.868719,0.138759,0.561224,0.021961,4715.561224,1041.967511,8.239905e+05,1.819878,-0.361238,2.0,429.000000
max,13021.230469,5365.638142,0.640165,1.000000,0.101965,7528.061224,5544.019437,7.502836e+06,25.507908,-0.151569,2.0,1046.000000


In [16]:
slice_df=voice_noise_df[voice_noise_df["type"]==2]
slice_df.loc[slice_df['RMS'].idxmin()]

RMS                       0.493964
SE                         2915.75
ZCR                      0.0293143
LEFR                             0
SF                      0.00198406
SRF                        7367.03
SC                         3977.55
BW                     5.20161e+06
NWPD                     -0.512627
RSE                      -0.283284
type                             2
name      speech-librivox-0150.wav
number                         788
Name: 236702, dtype: object

In [17]:
slice_df[slice_df['RMS']<103.6].sort_values(by=['RMS'],ascending=False) #-50dB

,RMS,SE,ZCR,LEFR,SF,SRF,SC,BW,NWPD,RSE,type,name,number
386308,103.584030,3844.206670,0.033752,0.295918,0.018916,3776.466837,263.515731,1.885769e+05,-2.123486,-0.318543,2,speech-us-gov-0241.wav,89
303818,103.533897,3157.873267,0.101069,0.000000,0.007104,5162.308673,561.912273,7.043432e+05,1.918406,-0.295715,2,speech-us-gov-0096.wav,229
199132,103.509537,3051.225503,0.130071,0.316327,0.008827,5558.035714,1018.195921,1.164952e+06,-2.819079,-0.317283,2,speech-librivox-0050.wav,281
301631,103.502007,3316.195603,0.089006,0.000000,0.007646,4860.650510,539.939378,4.950455e+05,2.330338,-0.292739,2,speech-us-gov-0092.wav,439
251239,103.466988,3579.663430,0.112945,0.000000,0.006647,3332.270408,735.273286,4.906727e+05,-2.875354,-0.298359,2,speech-us-gov-0004.wav,523
327072,103.340569,3450.350099,0.105632,0.632653,0.008268,3662.308673,834.551585,6.664655e+05,-0.646894,-0.359059,2,speech-us-gov-0137.wav,258
288668,103.264503,3287.543706,0.166760,0.326531,0.006601,4642.538265,1217.145337,8.567203e+05,2.097822,-0.328395,2,speech-us-gov-0069.wav,592
302701,103.189339,3399.239167,0.078630,0.000000,0.014733,4240.752551,503.332102,3.282388e+05,2.967351,-0.289400,2,speech-us-gov-0094.wav,310
233458,103.173180,3134.171755,0.147259,0.040816,0.005354,6293.367347,1088.597999,2.121370e+06,0.544089,-0.318783,2,speech-librivox-0142.wav,412
276602,103.172539,3367.763195,0.121820,0.316327,0.007524,4271.045918,854.785438,5.239398e+05,0.123551,-0.326095,2,speech-us-gov-0048.wav,460


In [18]:
slice_df.sort_values(by=['SE'],ascending=False).head(100)

,RMS,SE,ZCR,LEFR,SF,SRF,SC,BW,NWPD,RSE,type,name,number
202987,2033.158936,5365.638142,0.194887,0.377551,0.023882,4726.084184,1550.815161,810829.691157,1.682835,-0.574621,2,speech-librivox-0059.wav,356
378787,9194.875000,5295.393072,0.066567,0.040816,0.032225,3545.918367,539.578112,277861.795033,-3.763745,-0.421237,2,speech-us-gov-0227.wav,464
324679,6087.302734,5074.599190,0.047190,0.448980,0.028385,3429.528061,339.188329,195935.788766,-0.569363,-0.339124,2,speech-us-gov-0133.wav,311
332774,2760.376709,5047.551675,0.090256,0.469388,0.023936,3772.321429,624.032476,425533.836927,2.044248,-0.343267,2,speech-us-gov-0147.wav,28
328269,4048.552246,4994.142965,0.054566,0.459184,0.022604,3346.938776,413.188241,270384.415704,6.014378,-0.354209,2,speech-us-gov-0139.wav,281
358565,2353.612061,4971.523754,0.056066,0.255102,0.023229,3611.607143,402.339048,272880.553304,-1.928486,-0.371773,2,speech-us-gov-0192.wav,135
328230,5893.275391,4963.655864,0.045690,0.295918,0.018702,3477.040816,361.457641,111041.900318,1.402509,-0.342730,2,speech-us-gov-0139.wav,242
314422,2281.753662,4952.260842,0.106632,0.755102,0.013443,3303.890306,700.105378,560458.507589,1.234307,-0.339795,2,speech-us-gov-0115.wav,281
314124,5947.356934,4947.150698,0.103694,0.367347,0.013173,3453.443878,795.078925,368774.602598,-0.766976,-0.390189,2,speech-us-gov-0114.wav,583
392036,7500.341797,4933.441903,0.045315,0.244898,0.035296,3561.224490,334.443821,108949.299709,-2.467389,-0.375661,2,speech-us-gov-0251.wav,451


In [19]:
rms_filtered_df = voice_noise_df[voice_noise_df['RMS']>=103.6] #drop silent
rms_filtered_df.groupby('type').count()

,RMS,SE,ZCR,LEFR,SF,SRF,SC,BW,NWPD,RSE,name,number
type,,,,,,,,,,,,
0,151334,151334,151334,151334,151334,151334,151334,151334,151334,151334,151334,151334
1,20667,20667,20667,20667,20667,20667,20667,20667,20667,20667,20667,20667
2,213175,213175,213175,213175,213175,213175,213175,213175,213175,213175,213175,213175


In [20]:
rms_filtered_df.describe()

,RMS,SE,ZCR,LEFR,SF,SRF,SC,BW,NWPD,RSE,type,number
count,385176.000000,385176.000000,385176.000000,385176.000000,385176.000000,385176.000000,385176.000000,3.851760e+05,385176.000000,385176.000000,385176.000000,385176.000000
mean,3943.189627,3447.676950,0.106465,0.297430,0.018825,4444.452595,756.075715,6.771388e+05,0.268506,-0.357994,1.160553,216.347579
std,2646.163982,266.421265,0.061242,0.239607,0.009075,928.262018,492.212769,6.483314e+05,2.609559,0.076934,0.959463,173.941684
min,103.617767,2839.630946,0.000000,0.000000,0.001260,1718.750000,55.003685,6.713368e+03,-60.103344,-0.792863,0.000000,0.000000
25%,2009.436249,3265.883024,0.065692,0.040816,0.012743,3640.625000,442.242725,2.540518e+05,-1.174962,-0.410406,0.000000,74.000000
50%,3277.476807,3408.983609,0.094881,0.295918,0.017368,4276.147959,643.261823,4.825808e+05,0.315188,-0.355004,2.000000,171.000000
75%,5328.798340,3590.335183,0.133758,0.479592,0.023134,5113.520408,938.929237,8.833559e+05,1.784050,-0.295984,2.000000,329.000000
max,27775.876953,6964.074221,0.865992,0.989796,0.105078,7559.948980,7029.272341,8.526785e+06,52.321439,-0.139832,2.000000,1041.000000


In [21]:
music_dropped_df = rms_filtered_df[rms_filtered_df['type']>=1] #drop music

In [22]:
X=music_dropped_df.drop(['RMS','SE','type','name','number'], axis=1)
y=music_dropped_df['type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=666)

In [23]:
classifier = tree.DecisionTreeClassifier(max_depth = 3)
classifier = classifier.fit(X_train,y_train)

In [24]:
prediction = classifier.predict(X_test)
print(np.mean(np.equal(prediction,y_test).astype(np.float32)))

0.9651058316230774


## For testing new audio

In [25]:
from os.path import dirname,abspath,join
TEST_AUDIO_FOLDER = join(os.getcwd(),'src','data','testwav')
TEST_AUDIO_FOLDER

'C:\\Users\\tianr\\Programming\\Python\\Project Speaker Recog\\speaker_recognition_GMM_UBM\\src\\data\\testwav'

In [26]:
def create_dataset(DATA_FOLDER,WINDOW_LENGTH = 1,FRAME_LENGTH = 25):
    feature_name = "RMS,SE,ZCR,LEFR,SF,SRF,SC,BW,NWPD,RSE,type,name,number".split(",")
    features_dict = {feature:[] for feature in feature_name}

    for root, dirs, files in os.walk(DATA_FOLDER):
        for audio in files:
            if "noise" in audio or "music" in audio or "speech" in audio or "audio" in audio:
                print("****************************")
                print("reading:", audio)
                sampling_rate, sig = wav.read(join(root, audio))
                print("sampling rate:", sampling_rate, "signal length", len(sig))
                index = 0
                number = 0
                while index + (sampling_rate * WINDOW_LENGTH) < len(sig):
                    sample = sig[index:(index + (sampling_rate * WINDOW_LENGTH))]
                    ef = extract_features(sample, FRAME_LENGTH, sampling_rate)
                    rms, se, zcr, lefr, sf, srf, sc, bd, nwpd, rse = ef.return_()
                    features_dict["RMS"].append(rms)
                    features_dict["SE"].append(se)
                    features_dict["ZCR"].append(zcr)
                    features_dict["LEFR"].append(lefr)
                    features_dict["SF"].append(np.mean(sf))
                    features_dict["SC"].append(np.mean(sc))
                    features_dict["SRF"].append(np.mean(srf))
                    features_dict["BW"].append(np.mean(bd))
                    features_dict["NWPD"].append(np.mean(nwpd))
                    features_dict["RSE"].append(np.mean(rse))
                    features_dict["type"].append(audio.split("-")[0])
                    features_dict["name"].append(audio)
                    features_dict["number"].append(number)
                    number+=1
                    index += sampling_rate * WINDOW_LENGTH

    features_df=pd.DataFrame.from_dict(features_dict)
    features_df = features_df[feature_name]
    return features_df

In [27]:
test_df = create_dataset(TEST_AUDIO_FOLDER)

****************************
reading: audiotest08-06-2018-13-12-39.wav
sampling rate: 16000 signal length 84480
****************************
reading: audiotest08-06-2018-16-49-40.wav
sampling rate: 16000 signal length 84480
****************************
reading: audiotest08-06-2018-17-01-12.wav
sampling rate: 16000 signal length 84480
****************************
reading: audiotest08-06-2018-17-01-26.wav
sampling rate: 16000 signal length 84480
****************************
reading: audiotest08-06-2018-17-01-43.wav
sampling rate: 16000 signal length 84480
****************************
reading: audiotest08-06-2018-17-01-56.wav
sampling rate: 16000 signal length 84480
****************************
reading: audiotest08-06-2018-17-02-11.wav
sampling rate: 16000 signal length 84480
****************************
reading: audiotest08-06-2018-17-02-44.wav
sampling rate: 16000 signal length 84480


In [28]:
test_df.describe()

,RMS,SE,ZCR,LEFR,SF,SRF,SC,BW,NWPD,RSE,number
count,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,4.000000e+01,40.000000,40.000000,40.00000
mean,201.687123,3313.017930,0.138887,0.343367,0.019166,5698.445472,1025.381606,1.699581e+06,0.833485,-0.380689,2.00000
std,133.148892,300.870675,0.065652,0.270494,0.008853,794.982951,479.444434,1.355838e+06,1.684439,0.070517,1.43223
min,13.742748,2938.700814,0.026564,0.000000,0.003758,4008.928571,214.557554,7.480095e+04,-1.832598,-0.536759,0.00000
25%,99.066755,3027.137863,0.090584,0.109694,0.010970,5096.938776,649.228632,9.360658e+05,-0.241724,-0.429325,1.00000
50%,180.572105,3276.397727,0.121258,0.280612,0.020739,5773.437500,933.553075,1.218637e+06,0.562403,-0.379025,2.00000
75%,268.116066,3544.445905,0.177777,0.543367,0.024910,6255.102041,1301.169518,1.817482e+06,1.899510,-0.317243,3.00000
max,470.848419,4047.112678,0.276642,0.897959,0.036706,7083.545918,1944.187264,5.425501e+06,6.532763,-0.241685,4.00000


In [29]:
test_df=test_df.drop(['RMS','SE','type','name','number'], axis=1)
test_predictions = classifier.predict(test_df)
print(test_predictions)

[1 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 1 2 1 2 1 2 2 2 2 1 2 2 2
 2 2 2]


In [31]:
from sklearn.tree import _tree
import json

JSON_FILE_NAME=join(TEST_AUDIO_FOLDER,'tree_model.json')
feature_names = "ZCR,LEFR,SF,SRF,SC,BW,NWPD,RSE".split(",")
tree_ = classifier.tree_


feature_name = [
    feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
    for i in tree_.feature
]
print("def tree({}):".format(", ".join(feature_names)))

json_file = {}

def recurse(node, depth, json_file):
    indent = "  " * depth
    if tree_.feature[node] != _tree.TREE_UNDEFINED:
        name = feature_name[node]
        threshold = tree_.threshold[node]
        json_file["feature"] = name
        json_file["threshold"] = threshold
        json_file["decision"] = None
        print("{}if {} <= {}:".format(indent, name, threshold))
        try:
            temp = json_file["left"]
        except:
            json_file["left"] = {}
        recurse(tree_.children_left[node], depth + 1, json_file["left"])
        print("{}else:  # if {} > {}".format(indent, name, threshold))
        try:
            temp = json_file["right"]
        except:
            json_file["right"] = {}
        recurse(tree_.children_right[node], depth + 1, json_file["right"])
    else:
        print("{}return {}".format(indent, tree_.value[node]))
        json_file["decision"] = str(np.argmax(tree_.value[node]) == 1)
        json_file["threshold"] = 0.0
        json_file["feature"] = None
        json_file["left"] = None
        json_file["right"] = None
        return json_file

recurse(0, 1, json_file)
print(json.dumps(json_file, sort_keys=True, indent=4))
with open(JSON_FILE_NAME, "w") as file:
    json.dump(json_file, file)


def tree(ZCR, LEFR, SF, SRF, SC, BW, NWPD, RSE):
  if LEFR <= 0.07653061300516129:
    if SRF <= 3909.59814453125:
      if RSE <= -0.29504460096359253:
        return [[ 197. 1203.]]
      else:  # if RSE > -0.29504460096359253
        return [[1538.  741.]]
    else:  # if SRF > 3909.59814453125
      if SF <= 0.026406213641166687:
        return [[9704.  507.]]
      else:  # if SF > 0.026406213641166687
        return [[893. 688.]]
  else:  # if LEFR > 0.07653061300516129
    if RSE <= -0.2980712950229645:
      if SF <= 0.00462255347520113:
        return [[949. 547.]]
      else:  # if SF > 0.00462255347520113
        return [[  3230. 185795.]]
    else:  # if RSE > -0.2980712950229645
      if SRF <= 4435.1083984375:
        return [[ 395. 1461.]]
      else:  # if SRF > 4435.1083984375
        return [[1725.  884.]]
{
    "decision": null,
    "feature": "LEFR",
    "left": {
        "decision": null,
        "feature": "SRF",
        "left": {
            "decision": null,
   